# Notebook to handle data analysis of raw fltrace traces

## 0. Prerequisites
1. The traces obtained using `fltrace record -M <M_value> -L <L_value> -- program_name`. The parent folder of the traces is designed in the `PARENT_FOLDER_PATH` constant. Inside the parent folder should be several children folders named after benchmark/program traced. Inside each children folder should be grandchildren folders, one per each `(M,L)` pair. Then, inside each grandchildren folder should be the output files of fltrace, that is `fltrace-data-faults*.out`, `fltrace-data-procmaps*.out`, and (optionally) `fltrace-data-stats*.out`. 

    One example hierarchy is 

    ```
    parent/                                   
    ├── canneal/                              
    ┆   ├──200_50/                            
    ┆   ┆   ├──fltrace-data-faults-56090-1.out
    ┆   ┆   ├──fltrace-data-procmaps-56090.out
    ┆   ┆   └──fltrace-data-stats-56090.out   
    ┆   ├──200_100/                           
    ┆   ┆   ├──fltrace-data-faults-55308-1.out
    ┆   ┆   ├──fltrace-data-procmaps-55308.out
    ┆   ┆   └──fltrace-data-stats-55308.out   
    ┆   ├──200_150/                           
    ┆   ┆   ├──...                            
    ┆   └──200_200/                           
    ┆       ├──...                            
    └── fludanimate/                          
        ├──...                                
    ```
    See [the fltrace guide](../data/fltrace.md) to get information about how to get `fltrace` running, as well as how to extend/fix it to get more data. 

2. The objectdumps (`objdump -d /path/to/lib_or_executable_name > lib_or_executable_name`) of all relevant librairies accessed by the traced programs. The folder containing all the objdumps is designed in the `OBJDUMP_FOLDER_PATH` constant. That is, if a library/executable is present in one of the `procmaps` of one of the trace runs, its objdump should be in this directory.

    One example hierarchy is 

    ```
    objdumps/               
    ├── canneal             
    ├── fludanimate         
    ├── fltrace.so          
    ├── ld-linux-x86-64.so.2
    ├── libc.so.6           
    ├── libgcc_s.so.1       
    ├── libm.so.6           
    └── libstdc++.so.6.0.30 
    ```


### Constants

In [ ]:
PARENT_FOLDER_PATH = "../data/parsec_raw_out/"
OBJDUMP_FOLDER_PATH = "../data/objdumps/"